In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [3]:
topics = [
    'https://www.carsifu.my/news',
    'https://www.carsifu.my/car-reviews',
    'https://www.carsifu.my/bike-reviews',
]

In [6]:
cookies = {
    '_gid': 'GA1.2.1774589498.1688827808',
    '_pcid': '%7B%22browserId%22%3A%22lju4h0iqcndtfxp8%22%7D',
    'cX_P': 'lju4h0iqcndtfxp8',
    '_pctx': '%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAEzIEYOAWAZgAYATAHZ%2BADl68xwwRwEBWQSAC%2BQA',
    '__stgeo': 'IjAi',
    '__stbpnenable': 'MQ==',
    '__stdf': 'MA==',
    'cX_G': 'cx%3A1i0t3fg74zq623k3k9afr0x7u3%3A2nq99z7wrkb6x',
    '__stp': 'eyJ2aXNpdCI6InJldHVybmluZyIsInV1aWQiOiIwYzljYmIyZC02MTliLTQxMGItYTQzZS01YTg1N2EwMjhmODAifQ==',
    'XSRF-TOKEN': 'eyJpdiI6ImZGQlwvVGVHSkc4UE9CMjk1ZGlpcEpBPT0iLCJ2YWx1ZSI6InhrbUlrM2FKZlpROWU5XC91bnFESnJ3WTV1NzBidDdHNXFtaExFUHBNOThXQVBmTVZUTXFTTmVJTzB3bHRra0crNnZOMEIzYUpUcHQ2Tk5ZNVpjNERPdz09IiwibWFjIjoiYzBlOGQ4ZGYxMjIzMzRhZjZmYjg3NjcxYTVjNDJmZDg0OGE1ZjljZWU5NmRkZmFlYTI4MmI3YTZmOTQ5NWQ1MiJ9',
    'carsifu_session': 'eyJpdiI6IkZjUG1DYnQrTkcrS0k0c3ZkUjNxYUE9PSIsInZhbHVlIjoidmgzUm9xaGpqSFJQVkswb2J3SmtyQ3V4emlXbEI5REhuUlJ5R1drRmc3ZDk3UHQwOWVlNzB1TzhyVTYwZmJSamFIWGI2eWU3MGl3c3NCWlI0VnhYeHc9PSIsIm1hYyI6IjIzYWJmM2IyOTM4YzVjZjk4MmU1NzZjZGU5NTgxMjE0YzM4YmQwOGRjYmFiMDcyOTIzNDk4MDBhOTI5ZmM5NTIifQ%3D%3D',
    '_ga_QF13J11YBW': 'GS1.1.1688889128.2.1.1688889384.35.0.0',
    '_ga': 'GA1.1.2139008171.1688827808',
    '__sts': 'eyJzaWQiOjE2ODg4ODkxMjgzODEsInR4IjoxNjg4ODg5Mzg0MjgwLCJ1cmwiOiJodHRwcyUzQSUyRiUyRnd3dy5jYXJzaWZ1Lm15JTJGIiwicGV0IjoxNjg4ODg5Mzg0MjgwLCJzZXQiOjE2ODg4ODkxMjgzODEsInBVcmwiOiJodHRwcyUzQSUyRiUyRnd3dy5jYXJzaWZ1Lm15JTJGYmlrZS1yZXZpZXdzIiwicFBldCI6MTY4ODg4OTM2NzcxNywicFR4IjoxNjg4ODg5MzY3NzE3fQ==',
}

headers = {
    'authority': 'www.carsifu.my',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': '_gid=GA1.2.1774589498.1688827808; _pcid=%7B%22browserId%22%3A%22lju4h0iqcndtfxp8%22%7D; cX_P=lju4h0iqcndtfxp8; _pctx=%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAEzIEYOAWAZgAYATAHZ%2BADl68xwwRwEBWQSAC%2BQA; __stgeo=IjAi; __stbpnenable=MQ==; __stdf=MA==; cX_G=cx%3A1i0t3fg74zq623k3k9afr0x7u3%3A2nq99z7wrkb6x; __stp=eyJ2aXNpdCI6InJldHVybmluZyIsInV1aWQiOiIwYzljYmIyZC02MTliLTQxMGItYTQzZS01YTg1N2EwMjhmODAifQ==; XSRF-TOKEN=eyJpdiI6ImZGQlwvVGVHSkc4UE9CMjk1ZGlpcEpBPT0iLCJ2YWx1ZSI6InhrbUlrM2FKZlpROWU5XC91bnFESnJ3WTV1NzBidDdHNXFtaExFUHBNOThXQVBmTVZUTXFTTmVJTzB3bHRra0crNnZOMEIzYUpUcHQ2Tk5ZNVpjNERPdz09IiwibWFjIjoiYzBlOGQ4ZGYxMjIzMzRhZjZmYjg3NjcxYTVjNDJmZDg0OGE1ZjljZWU5NmRkZmFlYTI4MmI3YTZmOTQ5NWQ1MiJ9; carsifu_session=eyJpdiI6IkZjUG1DYnQrTkcrS0k0c3ZkUjNxYUE9PSIsInZhbHVlIjoidmgzUm9xaGpqSFJQVkswb2J3SmtyQ3V4emlXbEI5REhuUlJ5R1drRmc3ZDk3UHQwOWVlNzB1TzhyVTYwZmJSamFIWGI2eWU3MGl3c3NCWlI0VnhYeHc9PSIsIm1hYyI6IjIzYWJmM2IyOTM4YzVjZjk4MmU1NzZjZGU5NTgxMjE0YzM4YmQwOGRjYmFiMDcyOTIzNDk4MDBhOTI5ZmM5NTIifQ%3D%3D; _ga_QF13J11YBW=GS1.1.1688889128.2.1.1688889384.35.0.0; _ga=GA1.1.2139008171.1688827808; __sts=eyJzaWQiOjE2ODg4ODkxMjgzODEsInR4IjoxNjg4ODg5Mzg0MjgwLCJ1cmwiOiJodHRwcyUzQSUyRiUyRnd3dy5jYXJzaWZ1Lm15JTJGIiwicGV0IjoxNjg4ODg5Mzg0MjgwLCJzZXQiOjE2ODg4ODkxMjgzODEsInBVcmwiOiJodHRwcyUzQSUyRiUyRnd3dy5jYXJzaWZ1Lm15JTJGYmlrZS1yZXZpZXdzIiwicFBldCI6MTY4ODg4OTM2NzcxNywicFR4IjoxNjg4ODg5MzY3NzE3fQ==',
    'pragma': 'no-cache',
    'referer': 'https://www.carsifu.my/bike-reviews',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

In [14]:
r = requests.get(topics[1], cookies=cookies, headers=headers)
soup = BeautifulSoup(r.content, "lxml")
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if '/?page=' in a_.get('href', '')]
nums = []
for a_ in a:
    nums.append(int(a_.split('/?page=')[1]))
    
max(nums)

23

In [22]:
!rm -rf url
!mkdir url

In [23]:
def generator():
    while True:
        yield

for t in topics:
    print(t)
    t_ = t.split('my/')[1].replace('/', '-')
    index = 1
    
    r = requests.get(t, cookies=cookies, headers=headers)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if '/?page=' in a_.get('href', '')]
    nums = []
    for a_ in a:
        nums.append(int(a_.split('/?page=')[1]))
        
    for index in tqdm(range(1, max(nums) + 1, 1)):
    
        filename = os.path.join('url', f'{t_}-{index}.json')
        if os.path.exists(filename):
            index += 1
            continue
            
        url = f'{t}?page={index}'
        r = requests.get(url, cookies=cookies, headers=headers)
        soup = BeautifulSoup(r.content, "lxml")
        
        div = soup.find_all('div', {'class': 'col-lg-4 col-md-6 col-sm-6 col-xs-12 content_box'})
        a = [div_.find('a').get('href') for div_ in div]

        with open(filename, 'w') as fopen:
            json.dump(a, fopen)    

https://www.carsifu.my/news


100%|███████████████████████████████████████████████████████████████████████████████████████████| 997/997 [22:52<00:00,  1.38s/it]


https://www.carsifu.my/car-reviews


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:27<00:00,  1.21s/it]


https://www.carsifu.my/bike-reviews


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.08s/it]


In [21]:
url

'https://www.carsifu.my/news?page=61'